# Формирование таблицы словаря 'Marks' для базы данных.

### 1. Подготовка

In [15]:

import os
import time
import json
import datetime

import requests as rq
import pandas as pd
import numpy as np
import tqdm

from sqlalchemy import create_engine


### 2. Формирую переменные с нужными индексами марки и сотрудников, для формирования нужных тендеров.

Для этого подтягиваю из временных таблиц сформированных в других скриптах нужные данные.

In [ ]:

engine_pet = create_engine('mysql+mysqlconnector://root:''@''/pet_proect')

df_marks_ = pd.read_sql(
    'SELECT * FROM tmp_dict_marks', 
    con = engine_pet, 
    index_col ='_id'
).index.to_list()

df_users_ = pd.read_sql(
    'SELECT * FROM tmp_dict_users', 
    con = engine_pet, 
    index_col = '_id'
).index.to_list()

df_marks = '&marks=' + '&marks='.join(df_marks_)
df_users = '&users=' + '&users='.join(df_users_)


### 3. Формирование переменных для подключения

In [ ]:

TOKEN = 'Bearer '
API = '/api/tenders/v2/getlist'
URL = 'https://tenderplan.ru' + API
headers = {
    'Authorization': TOKEN,
    'Accept': 'application/json'
}


### 4. Формирование функции get запроса для цилка и функции преобразования времени

In [4]:

if __name__ == '__main__':
    def f_lst_tender(page_):
        '''Функция принимает в качестве аргумента номер страницы и выводит данные get запроса'''
        response = rq.get(
            URL + f'?page={str(page_)}' + df_marks + df_users,
            headers = headers
        ).json()
        return response['tenders']

def f_date_fr_time(x):
    '''Принимает UNIX-время и возвращате дату в формате datetime'''
    return datetime.datetime.fromtimestamp(x//1000)

def round_2(x):
    '''Функция округления числа'''
    return round(x, 2)


### 5. Загрузка данных через цикл WHILE (проверка на пустую страницу). 

На каждой итерации таймаут 0,08 секунд, так как в документации к API ограничение на кол-во запросов. Не более 250 запросов за 10 секунд или не более 800 запросов за 60 секунд

In [ ]:

lst_df = []
start_time = time.time()
count_page = 0

while f_lst_tender(count_page) != []:
    end_time = time.time()
    df_lst_tenders = f_lst_tender(count_page)
    lst_df = lst_df + df_lst_tenders
    count_page += 1
    time.sleep(0.08)
    
end_time = time.time()
del df_lst_tenders

print(f'Кол-во секунд: {round(end_time - start_time, 0)}')
print(f'Кол-во запросов: {len(lst_df) / 50}')
print(f'Кол-во запросов в секунду: {len(lst_df) / 50 / (end_time - start_time)}')
print(f'Кол-во секунд на запрос: {(end_time - start_time) / (len(lst_df) / 50)}')


Кол-во секунд: 131.0
Кол-во запросов: 152.8
Кол-во запросов в секунду: 1.168008878949939
Кол-во секунд на запрос: 0.8561578751858616


### 6. На данном этапе формируется несколько DataFrame:
1. df_participans - DataFrame участников тендера
2. df_marks_users - таблица фактов с пересечением марки, сотрудника и id тендера (несколько марок может быть на один тендер)
3. df_dict_tenders - справочник тенедров, были cJoinнены данные по стат показателям (мин, макс, ср, медиана по участникам тендера)

In [6]:

'''Создание всех DataFrame'''

'''Создание DataFrame участников'''
df_participants = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id', 
            'participants'
        ]
    )
    .set_index('_id')
    .explode('participants')
    .dropna(subset = 'participants')
)

df_participants = (
    pd.json_normalize(df_participants['participants'])
    .set_index(df_participants.index)
    .loc[
        :, 
        [
            'guid', 
            'name', 
            'price', 
            'region', 
            'score', 
            'winner'
        ]
    ]
)
    
df_participants = (
    df_participants
    .assign(
        **{
            'price': df_participants['price'].map(round_2),
            'score': df_participants['score'].map(round_2)
        }
    )
    .fillna(
        {
            'winner': np.nan,
            'guid': np.nan
        }
    )
)

'''Создание DataFrame стат. данных по участникам для справочника тендеров'''

agg_price_score = {
    'price': ['mean', 'std', 'min', 'max'],
    'score': ['mean', 'std', 'min', 'max']
}

df_stat_participants = (
    df_participants
    .groupby('_id')[['price', 'score']]
    .agg(agg_price_score)
    .round(2)
)

'''Перименование стобцов'''
df_stat_participants.columns = ['-'.join(x) for x in df_stat_participants.columns.values]

'''Создание DataFrame данных заказчика для справочника тендеров'''

df_customers = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'customers'
        ]
    )
    .set_index('_id')
    .explode('customers')
)

df_customers = (
    pd.json_normalize(df_customers['customers'])
    .set_index(df_customers.index)
    .astype(str)
    .groupby('_id')[['guid', 'name', 'region']]
    .agg(' '.join)
)

'''Создание DataFrame тендеров с денормализванными данными по меткам и сотрудникам'''

df_marks_users = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'marks',
            'users'
        ]
    )
    .set_index('_id')
    .explode('marks')
    .explode('users')
)

df_marks_users = df_marks_users.loc[
    (df_marks_users['marks'].isin(df_marks_)) & 
    (df_marks_users['users'].isin(df_users_))
]

'''Создание итого справочника по тендерам с данными заказчика и стат данными по участникам'''

df_dict_tenders = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'maxPrice',
            'status',
            'placingWay',
            'orderName',
            'publicationDate',
            'number',
            'region'
        ]
    )
    .set_index('_id')
    .join(
        df_customers, 
        rsuffix = '/customres'
    )
    .join(df_stat_participants)
)

df_dict_tenders = df_dict_tenders.assign(
    **{
        'publicationDate': df_dict_tenders['publicationDate'].map(f_date_fr_time)
    }
)


### 7. Загрузка данных во временные таблицы tmp

In [ ]:

(
    df_participants
    .reset_index()
    .to_sql(
        'tmp_participants', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)

time.sleep(1)

(
    df_marks_users
    .reset_index()
    .to_sql(
        'tmp_marks_users', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)

time.sleep(1)

(
    df_dict_tenders
    .reset_index()
    .to_sql(
        'tmp_dict_tenders', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)


-1

In [8]:

del df_stat_participants
del df_customers
del agg_price_score
del df_marks_
del df_users_


In [ ]:

df_participants.head()

,guid,name,price,region,score,winner
_id,,,,,,
68fb27225d48d39bb4df80e2,591ebd650640fd188e433f70,"ООО ГК ""ПЛАСТИК""",114347.79,52.0,0.72,NaN
68fb27225d48d39bb4df80e2,591ebdb10640fd1886538fe1,"ООО ""ВОДПОЛИМЕР""",114347.79,43.0,0.93,NaN
68fb27225d48d39bb4df80e2,591f14210640fd1a867a8520,"ООО ""УРАЛ-ТЕНДЕР""",105877.59,66.0,0.93,NaN
68fb27225d48d39bb4df80e2,591f1d8e0640fd1a867aa8df,"ООО ""ЦЕНТР ОПТОВОЙ ТОРГОВЛИ""",93172.28,66.0,0.93,NaN
68fb27225d48d39bb4df80e2,595bea7a2567a56d8b4793d5,"ООО ""МИР САНТЕХНИКИ""",67761.66,52.0,0.77,NaN


In [ ]:

df_marks_users.head()

,marks,users
_id,,
68fb27225d48d39bb4df80e2,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36
68fb1c795d48d39bb48fc427,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36
68fa39bf5d48d39bb421981f,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36
68fa38515d48d39bb414f640,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36
68fa206b5d48d39bb450815c,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36


In [ ]:

df_dict_tenders.head()

,maxPrice,status,placingWay,orderName,publicationDate,number,region,guid,name,region/customres,price-mean,price-std,price-min,price-max,score-mean,score-std,score-min,score-max
_id,,,,,,,,,,,,,,,,,,
68fb27225d48d39bb4df80e2,122818.00,1,22,Сантехнические товары,2025-10-24 07:00:00,0368300012625000207,73,58c7e6af0640fd10b742c63b,"ГУЗ ""ДГКБ Г. УЛЬЯНОВСКА""",73,98677.91,17973.57,67761.66,118582.90,0.85,0.09,0.72,0.93
68fb1c795d48d39bb48fc427,151600.00,1,15,Поставка расходных материалов для проведения к...,2025-10-23 07:00:00,0317100013125000111,22,58c7c3cf0640fd0b5eaa5409,ФКУ ИК-3 УФСИН РОССИИ ПО АЛТАЙСКОМУ КРАЮ,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68fa39bf5d48d39bb421981f,661840.00,1,22,Поставка радиаторов биметаллических для центра...,2025-10-23 07:00:00,32515333104,39,58c8032e0640fd10b74396dd,ГАУКОДО КОДЮЦЭКТ,39,527760.34,88336.86,410797.24,639017.93,0.92,0.04,0.87,0.98
68fa38515d48d39bb414f640,35661.80,1,22,Поставка обратных канализационных клапанов,2025-10-23 07:00:00,0372100049625002974,78,58c7ceb70640fd0b5eaacbad,"ФГБУ ""НМИЦ ИМ. В.А. АЛМАЗОВА"" МИНЗДРАВА РОССИИ",78,32218.59,2376.04,28283.50,34432.08,0.84,0.13,0.67,0.97
68fa206b5d48d39bb450815c,522041.75,1,22,Поставка радиаторов биметаллических,2025-10-23 07:00:00,32515332172,39,58c801560640fd10b192e0cd,ГАУ КО ПОО КСТ,39,422033.75,67422.01,324025.91,504040.31,0.93,0.03,0.90,0.99
